In [1]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import roc_curve
from sklearn.linear_model import RidgeCV, LassoCV
import statsmodels.api as sm

import pandas as pd
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.set_option('display.colheader_justify', 'left')

import warnings
warnings.filterwarnings('ignore')

In [13]:
homeprices_data = pd.read_excel('공통데이터.xlsx',sheet_name ='집값')
homeprices_data.set_index('time',inplace=True)
up_down_data = homeprices_data.copy()
for column in homeprices_data.columns[0:]:
    up_down_data[column] = up_down_data[column].diff().apply(lambda x: '1' if x > 0 else '0')
#상승 =1, 하락   =0
up_down_data = up_down_data.drop('2007-03-31')

up_down_data

,서울,부산,대구,인천,광주,대전,울산,경기,강원,충북,충남,전북,전남,경북,경남,제주
time,,,,,,,,,,,,,,,,
2007-06-30,0,1,0,1,0,0,1,0,0,0,1,1,1,1,1,1
2007-09-30,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,1
2007-12-31,1,1,0,1,1,1,0,1,0,1,1,1,1,0,1,1
2008-03-31,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
2008-06-30,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
2022-03-31,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1
2022-06-30,1,0,0,0,1,0,0,0,1,1,1,1,0,1,1,1


In [11]:
prices_data = pd.read_excel('공통데이터.xlsx',sheet_name ='물가수준')
prices_data.set_index('time',inplace=True)

# 물가수준 변화율
diff_price_data = prices_data.copy()
for column in prices_data.columns[0:]:
    diff_price_data[column] = diff_price_data[column].pct_change().apply(lambda x: round(x * 100, 2))
diff_price_data = diff_price_data.dropna()

diff_price_data

,서울,부산,대구,인천,광주,대전,울산,경기,강원,충북,충남,전북,전남,경북,경남,제주
time,,,,,,,,,,,,,,,,
2007-06-30,1.13,0.90,0.94,1.06,0.90,1.00,1.09,1.13,1.20,1.35,1.09,1.03,1.13,1.03,0.91,0.87
2007-09-30,0.57,0.76,0.67,0.70,0.93,0.86,0.92,0.73,0.64,0.61,0.57,0.61,0.86,0.80,0.67,0.61
2007-12-31,0.67,0.54,0.60,0.82,0.85,0.54,0.82,0.73,0.89,0.85,0.73,0.79,0.70,0.76,0.73,1.08
2008-03-31,1.20,1.29,1.35,1.38,1.25,1.26,1.40,1.35,1.48,1.31,1.47,1.29,1.13,1.35,1.55,1.52
2008-06-30,1.59,2.18,2.43,2.29,2.14,2.42,2.33,2.17,2.67,2.66,2.29,2.52,2.40,2.76,2.49,2.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,0.83,0.89,1.06,1.07,0.96,0.89,0.98,1.02,1.26,1.07,1.19,0.94,1.25,1.21,1.03,1.19
2022-03-31,1.39,1.56,1.56,1.58,1.40,1.60,1.51,1.52,1.72,1.65,1.68,1.48,1.52,1.67,1.76,1.78
2022-06-30,1.68,1.83,2.02,2.04,2.45,2.03,1.98,2.02,2.68,2.54,2.47,2.45,2.71,2.58,2.22,2.46


In [12]:
volatility_data = pd.read_excel('공통데이터.xlsx',sheet_name ='거래량 변화량')
volatility_data.set_index('time',inplace=True)

# 거래량 변화량 변화율
diff_volatility_data = volatility_data.copy()
for column in prices_data.columns[1:]:
    diff_volatility_data[column] = diff_volatility_data[column].pct_change().apply(lambda x: round(x * 100, 2))
diff_volatility_data = diff_volatility_data.drop(columns = '전국')
diff_volatility_data = diff_volatility_data.dropna()

diff_volatility_data

,서울,부산,대구,인천,광주,대전,울산,경기,강원,충북,충남,전북,전남,경북,경남,제주
time,,,,,,,,,,,,,,,,
2007-06-30,-154.1,-92.52,-107.92,-97.72,-102.05,-120.77,-137.57,-176.98,-102.94,-108.79,-87.60,-104.72,-100.67,-90.40,-85.87,19.92
2007-09-30,-104.4,114.74,-465.22,194.16,-30.59,-227.63,-45.17,-29.56,-1556.96,14.49,163.77,151.50,220.83,-93.12,127.98,-88.71
2007-12-31,-203.8,-141.90,-164.03,-332.23,590.68,-142.91,-0.28,129.33,-32.32,-422.15,-109.70,-27.65,-1137.66,-1695.00,-166.49,-448.33
2008-03-31,72.6,-207.14,-234.00,-121.10,-158.77,-495.58,-138.05,-107.92,57.89,-42.44,-721.19,-402.48,-3.00,0.47,-194.54,-74.64
2008-06-30,223.1,218.43,2.97,363.06,-92.69,103.88,329.74,930.41,163.01,564.51,211.46,51.57,35.48,-36.04,89.98,128.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,-166.9,125.61,18.27,-810.23,-473.71,67.78,6200.00,151.08,-118.99,-134.93,-214.80,-5860.00,192.66,-375.46,-157.07,-305.47
2022-03-31,-225.2,-24.55,-24.43,91.68,-39.67,50.96,-41.80,20.45,337.20,204.19,90.09,-31.08,5.22,20.47,-1.38,0.20
2022-06-30,-167.2,-3.07,-13.98,-17.11,7.57,-17.23,-45.15,-23.19,-25.30,-35.78,-24.40,-17.00,-26.22,-33.67,-3.05,20.96


In [15]:
common_data = pd.read_excel('공통데이터.xlsx',sheet_name ='공통데이터').dropna()
common_data.set_index('time',inplace=True)

common_data

,M2,rates,deltaM2
time,,,
2007-03-31,100.000000,6.163333,2.6
2007-06-30,101.912132,6.143333,1.9
2007-09-30,104.850878,6.380000,2.9
2007-12-31,108.372024,6.663333,3.4
2008-03-31,113.271747,6.833333,4.5
...,...,...,...
2021-12-31,310.897307,3.466667,3.0
2022-03-31,316.806167,3.856667,1.9
2022-06-30,320.679483,3.946667,1.2


In [17]:
# print(prices_data[region_data[1]])
region_data = prices_data.columns
print(region_data)
data_by_region = []
for i in region_data:
    tmp = common_data
    
    tmp_prices = pd.Series(prices_data[i])
    tmp_volatility = pd.Series(volatility_data[i])
    tmp_homeprices = pd.Series(homeprices_data[i])
    
    tmp_prices.name = tmp_prices.name+"물가"
    tmp_volatility.name = tmp_volatility.name+"거래량"
    tmp_homeprices.name = tmp_homeprices.name+"집값"
    tmp = pd.concat([tmp,tmp_prices,tmp_volatility,tmp_homeprices],axis = 1).dropna()
    data_by_region.append(tmp)

Index(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북',
       '전남', '경북', '경남', '제주'],
      dtype='object')


In [19]:
data_by_region[0].head()

,M2,rates,deltaM2,서울물가,서울거래량,서울집값
time,,,,,,
2007-03-31,100.000000,6.163333,2.6,100.0000,57.9,100.00000
2007-06-30,101.912132,6.143333,1.9,101.1290,-154.1,99.90685
2007-09-30,104.850878,6.380000,2.9,101.7092,-104.4,102.09190
2007-12-31,108.372024,6.663333,3.4,102.3873,-203.8,103.40940
2008-03-31,113.271747,6.833333,4.5,103.6114,72.6,107.36970


In [20]:
Data = pd.DataFrame()
intercept_data = 0
coef_data =0
parameter_data = 0

column = ['M2','rates','물가','거래량']
for i in range(len(data_by_region)):
    mlr = LinearRegression()
    지역 = region_data[i] 
    학습column = ['M2','rates',지역+'물가',지역+'거래량']
    정답column = [지역+'집값']
    
    학습data = data_by_region[i].loc[:,학습column]
    정답data = data_by_region[i].loc[:,정답column]
    mlr.fit(학습data, 정답data)
    
    
    intercept_data = pd.Series(mlr.intercept_)
    coef_data = pd.Series(np.hstack(mlr.coef_))
    score_data = pd.Series(mlr.score(학습data, 정답data))
    
    parameter_data = pd.DataFrame(pd.concat([intercept_data,coef_data,score_data],axis=0))
    parameter_data = parameter_data.T
    parameter_data.index = [지역]
    
    Data = pd.concat([Data,parameter_data],axis=0)
    
column = ['intercept','M2','rates','물가','거래량',"score"]    
Data.columns = column
Data

,intercept,M2,rates,물가,거래량,score
서울,466.662358,1.520099,0.789515,-5.156320,0.002935,0.966826
부산,-37.470529,0.352273,-5.888073,1.379239,-0.029245,0.951846
대구,224.457041,0.618283,-12.179727,-1.212281,-0.012920,0.931561
인천,176.304003,0.624360,6.239551,-1.571681,-0.017103,0.857653
광주,-8.215099,0.656379,-3.187499,0.472310,-0.000468,0.971880
대전,23.167320,0.518657,1.908144,0.099819,-0.022381,0.889041
울산,-3.011795,-0.021515,-6.893081,1.400924,-0.035106,0.843290
경기,173.491142,0.681584,6.146361,-1.765552,-0.007113,0.892412
강원,-47.376769,0.181635,-1.537711,1.327428,-0.030839,0.918188
충북,-127.142566,-0.211486,-4.060946,2.652134,-0.037230,0.797891


In [21]:
# Polynomial Regression

Data = pd.DataFrame()
intercept_data = 0
coef_data =0
parameter_data = 0

column = ['M2','rates','물가','거래량']
pvalues  = pd.DataFrame()
for i in range(len(data_by_region)):
    poly_reg = PolynomialFeatures(degree=2,include_bias = False)
    mlr = LinearRegression()
    
    지역 = region_data[i] 
    학습column = ['rates',지역+'거래량']
    정답column = [지역+'집값']
    
    학습data = data_by_region[i].loc[:,학습column]
    정답data = data_by_region[i].loc[:,정답column]
    
    M2data_poly = np.round(pd.DataFrame(poly_reg.fit_transform(data_by_region[i].loc[:,'M2'].values.reshape(-1,1))),2)
    물가data_poly = np.round(pd.DataFrame(poly_reg.fit_transform(data_by_region[i].loc[:,지역+'물가'].values.reshape(-1,1))),2)
    
    M2data_poly.index = 학습data.index
    M2data_poly.columns = ['M2','M2제곱']
    물가data_poly.index = 학습data.index
    물가data_poly.columns = ['물가','물가제곱']
    
    물가data_poly = pd.DataFrame(물가data_poly)
    학습data = pd.concat([학습data,M2data_poly,물가data_poly],axis=1)
    mlr.fit(학습data, 정답data)
    
    result = sm.OLS(정답data,학습data).fit()
    new_pvalue = pd.DataFrame(result.pvalues)
    new_pvalue.index = ['rates','거래량','M2','M2제곱','물가','물가제곱']
    new_pvalue.columns = [지역]
    pvalues = pd.concat([new_pvalue, pvalues],axis=1)
    
    
    rmse_data = pd.Series(mean_squared_error(정답data, mlr.predict(학습data)))
    score_data = pd.Series(mlr.score(학습data, 정답data))
    parameter_data = pd.DataFrame(pd.concat([ rmse_data, score_data],axis=0))
    parameter_data = parameter_data.T
    parameter_data.index = [지역]
    Data = pd.concat([Data,parameter_data],axis=0)
    
column = ["MSE","score"]  
Data.columns = column
pvalues = round(pvalues.T,2)
Data

,MSE,score
서울,50.068738,0.976987
부산,92.047193,0.954144
대구,107.357905,0.935304
인천,49.938645,0.906624
광주,56.086077,0.976729
대전,34.323426,0.971516
울산,76.925898,0.855267
경기,36.027587,0.937598
강원,55.839378,0.926928
충북,83.873151,0.823598
